# This is to build a hexagon level estimate using the baysian model

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sys
ROOT_DIR = "../../"
sys.path.append(ROOT_DIR)
from utils.citymeta import load_all
N = 7 # 13
RES =res = 9 # need to test the 8 later
N_CAT = 27
import os
# prefix = "built_env"
# prefixfull = "_built_environment"
# merge both prefix
prefix_ls = ["_built_environment", ""]
ROOT = "D:/Dropbox (Personal)/Personal Work/_Projects2023"
CURATED_FOLDER = f"{ROOT}/01_city-never-was/_data/_curated/"
EXPORT_FOLDER = f"{ROOT}/01_city-never-was/_data/_curated/c_analysis"

In [2]:
prefix = "_built_environment"
hex_path = os.path.join(EXPORT_FOLDER,f"c_hex{prefix}_full_cluster={N}_ncat={N_CAT}.csv")
hex_df = pd.read_csv(hex_path)
hex_df.head()

C:\Users\yuanzfan\AppData\Local\Temp\ipykernel_10400\1546773337.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  hex_df = pd.read_csv(hex_path)


,hex_id,city_lower,total_flux,mean_flux,flux_n_points,City,country_clean,State/Province,urban_pop,Metro,...,obj_person,obj_train,obj_truck,person_exposure,bicycle_exposure,motorcycle_exposure,length_intersection_meter,Continent,waze_total_accident_city,include
0,8975292c127ffff,accra,8.835611,0.883561,10,Accra,Ghana,Greater Accra,2605000,NaN,...,4.800000,0.0,3.60,1.000000,1.0,0.0,259.370082,Africa,0.0,0
1,8975292c803ffff,accra,8.835611,0.883561,10,Accra,Ghana,Greater Accra,2605000,NaN,...,0.300000,0.0,0.00,1.000000,0.0,0.0,2507.096480,Africa,0.0,0
2,8975292c80bffff,accra,8.835611,0.883561,10,Accra,Ghana,Greater Accra,2605000,NaN,...,0.950000,0.0,0.05,0.578947,0.0,0.0,2516.047969,Africa,0.0,0
3,8975292c817ffff,accra,8.835611,0.883561,10,Accra,Ghana,Greater Accra,2605000,NaN,...,1.285714,0.0,0.00,0.944444,1.0,0.0,2955.782641,Africa,0.0,0
4,8975292c81bffff,accra,8.835611,0.883561,10,Accra,Ghana,Greater Accra,2605000,NaN,...,0.875000,0.0,0.00,0.714286,0.0,0.0,2328.091908,Africa,0.0,0


In [3]:
continent_sel = ['Asia', 'Europe', 'North America',  'South America']
hex_df.columns = [x.lower() for x in hex_df.columns]

In [4]:
continent = "North America"
df = hex_df[hex_df['continent'] == continent].copy()
print(df.shape)

(49232, 80)


In [5]:
import h3
import pandas as pd

# Suppose df is your dataframe
adjacency = {
    h: h3.k_ring(h, 1) - {h}  # immediate neighbors
    for h in df['hex_id']
}
print(len(adjacency.keys()))

49232


In [9]:
from libpysal.weights import W

# Make mapping from h3_id to row index
h3_index = {h: i for i, h in enumerate(df['hex_id'])}

# Create weight dictionary with integer indices
neighbors = {
    h3_index[h]: [h3_index[n] for n in adjacency[h] if n in h3_index]
    for h in df['hex_id']
}

w = W(neighbors)

C:\Users\yuanzfan\AppData\Local\Temp\ipykernel_10400\3658422626.py:12: UserWarning: The weights matrix is not fully connected: 
 There are 544 disconnected components.
 There are 301 islands with ids: 17, 51, 52, 146, 149, 150, 187, 382, 399, 439, 441, 507, 537, 551, 562, 603, 651, 896, 897, 905, 914, 915, 920, 921, 922, 924, 925, 926, 933, 938, 940, 941, 942, 944, 949, 951, 954, 955, 960, 965, 969, 970, 974, 977, 980, 984, 992, 994, 1001, 1002, 1003, 1008, 1010, 1014, 1021, 1022, 1030, 1031, 1032, 1034, 1038, 1041, 1042, 1043, 1044, 1045, 1046, 1047, 1048, 1049, 1050, 1051, 1052, 1056, 1057, 1058, 1059, 1069, 1075, 1079, 1081, 1082, 1088, 1089, 1090, 1091, 1094, 1106, 1110, 1112, 1116, 1117, 1122, 1123, 1127, 1132, 1133, 1136, 1137, 1154, 1155, 1156, 1160, 1161, 1162, 1163, 1164, 1171, 1173, 1176, 1177, 1178, 1182, 1191, 1195, 1196, 1197, 1198, 1199, 1203, 1204, 1205, 1206, 1208, 1210, 1213, 1216, 1221, 1222, 1223, 1225, 1230, 1234, 1241, 1252, 1255, 1258, 1259, 1262, 1265, 1266, 1267

In [10]:
import pymc as pm
import numpy as np

ModuleNotFoundError: No module named 'pymc'